# k-Nearest Neighbors Classifier

In this notebook, you will implement your own k-nearest neighbors (k-NN) algorithm for the classification problem. You are supposed to learn:

* How to prepare the dataset for "training" and testing of the model.
* How to implement k-nearest neighbors classification algorithm.
* How to evaluate the performance of your classifier.

**Instructions:**

* Read carefuly through this notebook. Be sure you understand what is provided to you, and what is required from you.
* Place your code only in sections annotated with `### START CODE HERE ###` and `### END CODE HERE ###`.
* Use comments whenever the code is not self-explanatory.
* Submit an executable notebook (`*.ipynb`) with your solution to BlackBoard.

Enjoy :-)

## Packages

Following packages is all you need. Do not import any additional packages!

* [Pandas](https://pandas.pydata.org/) is a library providing easy-to-use data structures and data analysis tools.
* [Numpy](http://www.numpy.org/) library provides support for large multi-dimensional arrays and matrices, along with functions to operate on these.

In [1]:
import pandas as pd
import numpy as np

## Problem

You are given a dataset `mushrooms.csv` with characteristics/attributes of mushrooms, and your task is to implement and evaluate a k-nearest neighbors classifier able to say whether a mushroom is poisonous or edible based on its attributes.

## Dataset

The dataset of mushroom characteristics is freely available at [Kaggle Datasets](https://www.kaggle.com/uciml/mushroom-classification) where you can find further information about the dataset. It consists of 8124 mushrooms characterized by 23 attributes (including the class). Following is the overview of attributes and values:

* class: edible=e, poisonous=p
* cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
* cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
* cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
* bruises: bruises=t,no=f
* odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
* gill-attachment: attached=a,descending=d,free=f,notched=n
* gill-spacing: close=c,crowded=w,distant=d
* gill-size: broad=b,narrow=n
* gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
* stalk-shape: enlarging=e,tapering=t
* stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
* stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* veil-type: partial=p,universal=u
* veil-color: brown=n,orange=o,white=w,yellow=y
* ring-number: none=n,one=o,two=t
* ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
* spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
* population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
* habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

Let's load the dataset into so called Pandas dataframe.

In [2]:
mushrooms_df = pd.read_csv('mushrooms.csv')

Now we can take a closer look at the data.

In [3]:
mushrooms_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


You can also print an overview of all attributes with the counts of unique values.

In [4]:
mushrooms_df.describe().T

,count,unique,top,freq
class,8124,2,e,4208
cap-shape,8124,6,x,3656
cap-surface,8124,4,y,3244
cap-color,8124,10,n,2284
bruises,8124,2,f,4748
odor,8124,9,n,3528
gill-attachment,8124,2,f,7914
gill-spacing,8124,2,c,6812
gill-size,8124,2,b,5612
gill-color,8124,12,b,1728


The dataset is pretty much balanced. That's a good news for the evaluation.

## Dataset Preprocessing

As our dataset consist of nominal/categorical values only, we will encode the strings into integers which will allow us to use similiraty measures such as Euclidean distance.

In [5]:
def encode_labels(df):
    import sklearn.preprocessing
    encoder = {}
    for col in df.columns:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])
        encoder[col] = le
    return df, encoder    

mushrooms_encoded_df, encoder = encode_labels(mushrooms_df)

In [6]:
mushrooms_encoded_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


## Dataset Splitting

Before we start with the implementation of our k-nearest neighbors algorithm we need to prepare our dataset for the "training" and testing.

First, we divide the dataset into attributes (often called features) and classes (often called targets). Keeping attributes and classes separately is a common practice in many implementations. This should simplify the implementation and make the code understandable.

In [7]:
X_df = mushrooms_encoded_df.drop('class', axis=1)  # attributes
y_df = mushrooms_encoded_df['class']  # classes
X_array = X_df.as_matrix()
y_array = y_df.as_matrix()

/Users/kristoffergjerde/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kristoffergjerde/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


And this is how it looks like.

In [8]:
print('X =', X_array)
print('y =', y_array)

X = [[5 2 4 ... 2 3 5]
 [5 2 9 ... 3 2 1]
 [0 2 8 ... 3 2 3]
 ...
 [2 2 4 ... 0 1 2]
 [3 3 4 ... 7 4 2]
 [5 2 4 ... 4 1 2]]
y = [1 0 0 ... 0 1 0]


Next, we need to split the attributes and classes into training sets and test sets.

**Exercise:**

Implement the holdout splitting method with shuffling.

In [9]:
from random import seed
from random import randrange

def train_test_split(X, y, test_size=0.2):
    """
    Shuffles the dataset and splits it into training and test sets.
    
    :param X
        attributes
    :param y
        classes
    :param test_size
        float between 0.0 and 1.0 representing the proportion of the dataset to include in the test split
    :return
        train-test splits (X-train, X-test, y-train, y-test)
    """
    ### START CODE HERE ###
    # Shuffle first
    c = list(zip(X, y))
    test = list()
    test_size = test_size * len(c)
    copy = list(c)
    while len(test) < test_size:
        index = randrange(len(copy))
        test.append(copy.pop(index))
    X_train, y_train = zip(*copy)
    X_test, y_test =  zip(*test)
    
    
    ### END CODE HERE ###
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

Let's split the dataset into training and validation/test set with 67:33 split.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, 0.33)

In [11]:
print('X_train =', X_train)
print('y_train =', y_train)
print('X_test =', X_test)
print('y_test =', y_test)

X_train = [[0 2 8 ... 3 2 3]
 [5 3 9 ... 2 2 1]
 [0 2 8 ... 2 2 3]
 ...
 [5 2 4 ... 0 4 2]
 [2 2 4 ... 0 1 2]
 [5 2 4 ... 4 1 2]]
y_train = [0 0 0 ... 0 0 0]
X_test = [[2 0 2 ... 2 5 0]
 [5 3 2 ... 7 4 4]
 [5 3 9 ... 2 3 1]
 ...
 [5 0 3 ... 1 4 0]
 [5 0 2 ... 3 5 0]
 [2 3 3 ... 1 5 1]]
y_test = [0 1 0 ... 1 0 1]


A quick sanity check...

In [12]:
assert len(X_train) == len(y_train)
assert len(y_train) == 5443
assert len(X_test) == len(y_test)
assert len(y_test) == 2681

## Algorithm

The k-nearest neighbors algorithm doesn't require a training step. The class of an unseen sample is deduced by comparison with samples of known class.

**Exercise:**

Implement the k-nearest neighbors algorithm.

In [13]:
# Use this section to place any "helper" code for the `knn()` function.

### START CODE HERE ###
def class_counts(y):
    """
    A function for counting the occurrence of the different labels in y
    (or attributes if col used)
    y: numpy.array[]
    """
    counts = {}
    for label in y:
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

### END CODE HERE ###

In [14]:
import operator
def knn(X_true, y_true, X_pred, k=5):
    """
    k-nearest neighbors classifier.
    
    :param X_true
        attributes of the groung truth (training set)
    :param y_true
        classes of the groung truth (training set)
    :param X_pred
        attributes of samples to be classified
    :param k
        number of neighbors to use
    :return
        predicted classes
    """
    ### START CODE HERE ### 
    

    
    count = 0 # Just for printing
    y_pred = []
    # For each of the rows we are going to predict we need to calculate the distance between the row
    # and all the others in X_true
    for test_row_index in range(len(X_pred)):
        
        matrix = []
        for true_row_index in range(len(X_true)):
            dist = np.linalg.norm(X_true[true_row_index] - X_pred[test_row_index])
            matrix.append((true_row_index, dist))
        
        # Find the k nodes with the shortes distance
        sorted_matrix = sorted(matrix, key=operator.itemgetter(1))
        k_nearest = sorted_matrix[:k]
        
        # Get their labels
        true_labels = []
        for node in k_nearest:
            index = node[0]
            true_labels.append(y_true[index])
        
        # Choose the label with highest occurence
        stats = class_counts(true_labels)
        predict = max(stats.items(), key=operator.itemgetter(1))[0]
        y_pred.append(predict)
        
        # Just for printing
        if count % 50 == 0:
            print("Predicted row: ", count )
        count += 1

    ### END CODE HERE ### 
    return np.array(y_pred)

In [15]:
y_hat = knn(X_train, y_train, X_test, k=5)
y_hat

Predicted row:  0
Predicted row:  50
Predicted row:  100
Predicted row:  150
Predicted row:  200
Predicted row:  250
Predicted row:  300
Predicted row:  350
Predicted row:  400
Predicted row:  450


KeyboardInterrupt: 

First ten predictions of the test set.

In [ ]:
y_hat[:10]

## Evaluation

Now we would like to assess how well our classifier performs.

**Exercise:**

Implement a function for calculating the accuracy of your predictions given the ground truth and predictions.

In [21]:
def evaluate(y_true, y_pred):
    """
    Function calculating the accuracy of the model given the ground truth and predictions.
    
    :param y_true
        true classes
    :param y_pred
        predicted classes
    :return
        accuracy
    """
    ### START CODE HERE ###
    n = len(y_true)
    count = 0
    for x in range(n):
        if y_true[x] == y_pred[x]:
            count += 1
    ### END CODE HERE ### 
    return count/n

In [ ]:
accuracy = evaluate(y_test, y_hat)
print('accuracy =', accuracy)

How many items where misclassified?

In [ ]:
print('misclassified =', sum(abs(y_hat - y_test)))

How balanced is our test set?

In [ ]:
np.bincount(y_test)

If it's balanced, we don't have to be worried about objectivity of the accuracy metric.

---

Congratulations! At this point, hopefully, you have successufuly implemented a k-nearest neighbors algorithm able to classify unseen samples with high accuracy.

✌️

# Metoder Data

In [16]:
# Read from file
df = pd.read_csv('metoder.txt', sep="\t", header=None)
df.columns = ["One", "Two", "Three","Four","Five","Six" ,"Seven","class"]
df.head()

,One,Two,Three,Four,Five,Six,Seven,class
0,1,1,2,2,1,1,1,1
1,1,2,2,1,1,1,1,1
2,1,2,1,2,2,2,2,1
3,1,1,2,2,2,1,1,1
4,2,2,1,1,2,2,1,2


In [26]:
# Split in features and labels
X_df = df.drop('class', axis=1)  # attributes
y_df = df['class']  # classes

y_df = y_df.apply(lambda x: x-1)
X_array = X_df.as_matrix()
y_array = y_df.as_matrix()
print(X_array)
print(y_array)

[[1 1 2 2 1 1 1]
 [1 2 2 1 1 1 1]
 [1 2 1 2 2 2 2]
 [1 1 2 2 2 1 1]
 [2 2 1 1 2 2 1]
 [1 2 2 1 2 1 2]
 [1 1 1 1 1 1 2]
 [1 2 1 1 1 2 1]
 [2 1 2 1 1 1 2]
 [1 2 2 2 2 1 1]
 [2 1 1 1 2 2 1]
 [1 1 2 2 1 2 1]
 [1 2 2 2 1 1 1]
 [2 2 2 1 2 1 2]
 [1 2 2 1 1 2 2]
 [2 1 2 1 1 2 1]
 [2 1 2 1 1 2 2]
 [1 2 2 2 2 2 2]
 [2 1 1 2 2 1 2]
 [2 2 2 2 1 1 1]
 [2 1 1 2 1 1 1]
 [1 1 1 2 2 1 1]
 [1 1 1 1 1 1 1]
 [2 1 2 1 2 1 2]
 [2 1 1 1 2 1 2]
 [2 2 2 1 1 1 1]
 [2 2 1 1 1 1 1]
 [1 2 2 1 1 1 2]
 [1 2 2 2 2 2 1]
 [2 2 2 1 2 2 1]
 [2 2 2 1 1 2 2]
 [1 2 1 1 1 1 1]
 [1 1 1 2 1 1 2]
 [2 1 2 2 1 2 2]
 [1 2 1 1 1 1 2]
 [2 2 2 2 1 1 2]
 [2 2 1 2 2 1 2]
 [1 2 2 2 2 1 2]
 [2 2 2 2 2 2 1]
 [1 2 1 2 2 1 2]
 [1 2 1 1 2 2 2]
 [1 1 2 1 2 2 2]
 [2 1 2 2 1 1 1]
 [1 1 2 2 1 2 2]
 [2 1 1 1 1 2 2]
 [2 2 1 1 1 2 1]
 [2 1 2 2 1 1 2]
 [1 1 2 1 1 1 2]
 [2 2 2 1 1 1 2]
 [2 1 1 1 1 2 1]
 [2 1 1 1 2 2 2]
 [1 1 1 2 2 1 2]
 [2 2 2 2 2 1 2]
 [1 1 2 1 1 1 1]
 [1 1 2 1 1 2 1]
 [1 1 2 1 2 1 1]
 [1 2 1 2 1 2 2]
 [1 2 1 2 1 2 1]
 [2 1 1 2 1 2 

/Users/kristoffergjerde/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/kristoffergjerde/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [27]:
# Split in training and data
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, 0.33)
print('X_train =', len(X_train))
print('y_train =', y_train)
print('X_test =', len(X_test))
print('y_test =', y_test)

X_train = 85
y_train = [0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1
 1 1 1 0 0 0 0 0 0 1 0]
X_test = 43
y_test = [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0
 1 0 0 0 0 0]


In [32]:
y_hat = knn(X_train, y_train, X_test, k=9)
y_hat

Predicted row:  0


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [33]:
accuracy = evaluate(y_test, y_hat)
print('accuracy =', accuracy)

accuracy = 0.8372093023255814


In [34]:
print('misclassified =', sum(abs(y_hat - y_test)))

misclassified = 7


In [35]:
np.bincount(y_test)

array([34,  9])